In [1]:
import sqlite3
from collections import Counter
import pandas as pd
import nltk
import plotly.express as px

In [2]:
query = 'select * from daily_top20_artists;'
query_artists = 'select * from artists;'

In [3]:
conn = sqlite3.connect('spotify.db')

cursor = conn.cursor()

In [4]:
cursor.execute(query_artists)
blob = cursor.fetchall()

In [5]:
genres_rawcount = [i[4] for i in blob]

In [6]:
genre_blob = ' '.join(genres_rawcount)

In [7]:
genre_wordlist = nltk.word_tokenize(genre_blob)

In [8]:
#no NLTK
raw_grams = Counter(genres_rawcount).most_common()

#top 10 bigrams
bigrams = Counter(list(nltk.bigrams(genre_wordlist))).most_common(10)

#top 10 unigrams
unigrams = list(enumerate(Counter(genre_wordlist).most_common(10)))

## Plotly 

In [26]:
#Raw_grams
df_raw = pd.DataFrame(raw_grams, columns = ['genre','count'])

blob = df_raw['genre'] == 'None'

df_raw = df_raw.loc[~blob]

In [29]:
fig = px.bar(df_raw.iloc[:10], x='genre', y='count')
fig.show()

In [10]:
#top 10 unigrams
unigrams = Counter(genre_wordlist).most_common(10)
df_uni = pd.DataFrame(unigrams, columns = ['genre','count'])

In [11]:
fig = px.bar(df_uni, x='genre', y='count')
fig.show()

## Pandas

In [12]:
cols = ['index', 'position', 'art_id', 'art_name', 'popularity', 'followers', 'day_of_chart']

In [13]:
chart_appearances = df.groupby(['art_name'])['position'].count()

NameError: name 'df' is not defined

In [ ]:
chart_appearances.sort_values().tail(20)